In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

In [2]:
#Fake data
df = pd.DataFrame({})
regions = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "L"]

for i in range(len(regions)):
  df.insert(i, column=regions[i], value=np.random.laplace(size=int(1e6)))

df

,A,B,C,D,E,F,G,H,I,L
0,-1.583431,-5.690821,1.393852,-0.632255,-2.630407,-0.801062,-0.603722,4.247088,-2.706628,-0.871662
1,2.440606,1.564009,-0.306757,0.170099,0.464557,-0.527898,0.079379,-0.378123,-1.996626,0.223857
2,-0.323044,-3.833118,-0.457599,0.974531,-0.688972,-2.723195,-0.638428,0.420517,-0.522992,2.019345
3,0.073895,0.476307,-1.172540,-2.569375,0.877282,-0.273370,1.239567,0.474450,0.528610,-1.087441
4,-1.521224,0.017516,-0.060925,3.405464,0.402572,-0.167009,-1.430690,0.635711,1.167589,-0.762382
...,...,...,...,...,...,...,...,...,...,...
999995,-0.581672,-0.386457,-0.210547,0.062117,1.552010,-0.347576,-0.465095,1.268499,-0.205128,0.980443
999996,0.691099,1.152085,-0.619477,0.401816,0.426875,-2.261062,-0.232320,-0.276161,-1.918823,0.131867
999997,1.009230,-1.142433,1.508793,-0.727750,0.285071,0.007690,-0.301045,-3.682997,-1.212841,2.577396
999998,0.060582,1.020180,-0.504067,-0.437576,1.023171,-2.896188,-2.502269,1.321322,2.289088,0.147100


In [3]:
df.shape

(1000000, 10)

$FC(t)$: correlation between columns in a given window of time $\tau$

In [4]:
#t and tau are indices, could be necessary some conversion
def FC(t, tau, data):
  #Define the time window
  data_window = data[int((t-tau/2)):int((t+tau/2))]

  #.corr() method returns a Pandas DataFrame, .values makes it a Numpy Array (convenient for later consistency)
  #Lost information about names of DafaFrame columns 
  FC_dataframe = data_window.corr(method="pearson")
  FC_array = FC_dataframe.values

  return FC_array

$dFC(t_1, t_2)$: measure of correlation between two FC

$dFC(t_1, t_2) = corr(UpperTri(FC(t_1)), UpperTri(FC(t_2))$

In [5]:
def dFC(FC_1, FC_2):
  #Get the upper triangular matrix of FC
  UpperTri_1 = FC_1[np.triu_indices(FC_1.shape[1], k=1)]
  UpperTri_2 = FC_2[np.triu_indices(FC_2.shape[1], k=1)]

  #Pearson correlation
  rho = np.corrcoef(UpperTri_1, UpperTri_2)[0, 1]
  return rho

$v_{dFC; \tau}(t)$: speed in dFC by measure of correlation between FC separated by $\tau$

$v_{dFC; \tau}(t)=1-dFC(t, t+\tau)$

In [6]:
#Note that it is being used the same tau both for FC window and speed
def v_dFC(t, tau, data):
  return  1 - dFC(FC(t, tau, data), FC(t+tau, tau, data))

$dFC$ stream: sequence of FC(t) by sliding a fixed time window $\tau$ with $0$ overlap (!)

return an array of correlation matrices

**range(100) needs to be updated!!**

In [ ]:
def dFC_stream(data, tau):
  stream = np.zeros(shape=(1, data.shape[1], data.shape[1]))
  
  #range(100) needs to be updated
  for i in range(100):
    stream = np.append(stream, [FC(i+tau/2, tau, data)], axis=0)
  
  return stream[1:]

$v_{stream}$: returns $v_{dFC; \tau}$ distribution for a given dFC stream

$\tau$ is fixed by the dFC stream

In [8]:
def v_stream(dFC_stream):
  v_array = np.array([])

  for i in range(dFC_stream.shape[0]-1):
    v = 1 - dFC(dFC_stream[i], dFC_stream[i+1])
    v_array = np.append(v_array, v)
  
  return v_array

$dFC$ matrix: similarity between FC networks

$(dFC_{matrix})_{ij} = dFC(t_i, t_j) \quad \forall i, j$

In [ ]:
def dFC_matrix(dFC_stream):
  #Size of dFC_matrix: number of FC in dFC_stream
  num_FC = dFC_stream.shape[0]
  matrix = np.zeros(shape=(num_FC, num_FC))

  #Fill lower triangula matrix
  for i in range(num_FC):
    for j in range(i+1):
      matrix[i, j] = dFC(dFC_stream[i], dFC_stream[j])
  
  #Get the symmetric matrix
  matrix = matrix + matrix.T - np.diag(matrix)

  return matrix

Pooling: $v_{dFC, \tau}$ distribution is similar for similar $\tau$: increase the statistic

In [ ]:
def pooled_v_stream(data, tau_min, tau_max):
  v_array = np.array([])

  for tau in range(tau_max - tau_min + 1):
    #Get dFC stream and v distribution at fixed tau
    stream_dFC = dFC_stream(data, tau)
    stream_v = v_stream(stream_dFC)

    v_array = np.append(v_array, stream_v)
  
  return v_array

t_SNE: 2D visualization of similarity evolution (stochastic walk) of UpperTri(FC(t)): dimensionality reduction

In [ ]:
def tSNE_evolution(dFC_stream):
  #Create empty matrix of UpperTri: shape = (number of FC in dFC stream, len(UpperTri))
  dim_UpperTri = len(dFC_stream[0][np.triu_indices(dFC_stream[0].shape[1], k=1)])
  UpperTri_matrix = np.zeros(shape=(dFC_stream.shape[0], dim_UpperTri))

  #Fill UpperTri_matrix  
  for i in range(dFC_stream.shape[0]):
    #Get FC(t_i) and UpperTri(FC(t_i)) vector
    FC = dFC_stream[i]
    UpperTri = FC[np.triu_indices(FC.shape[1], k=1)]
    
    UpperTri_matrix[i] = UpperTri

  #Points in 2D space
  embedded_points = TSNE(n_components=2, perplexity=30.0, early_exaggeration=4.0, method="exact").fit_transform(UpperTri_matrix)
  
  #Isolate x, y
  feature1 = embedded_points[:, 0]
  feature2 = embedded_points[:, 1]
  
  return feature1, feature2